<a href="https://colab.research.google.com/github/MichaelR-DS/Retail_Customer_Segmentation/blob/main/retail_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
import pandas as pd
import numpy as np

%matplotlib inline

In [103]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [104]:
df = pd.read_csv('/content/drive/MyDrive/online_retail_data_analysis/retail_data_clean.csv', index_col=[0])

In [105]:
df['TotalSpent'] = df['Quantity']*df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSpent
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [106]:
df['InvoiceNo'].astype('string').str[0].isin(['C', 'c']).any() #No cancellations

False

In [107]:
df[['Quantity', 'TotalSpent']].describe()

,Quantity,TotalSpent
count,397884.000000,397884.000000
mean,12.988238,22.397000
std,179.331775,309.071041
min,1.000000,0.001000
25%,2.000000,4.680000
50%,6.000000,11.800000
75%,12.000000,19.800000
max,80995.000000,168469.600000


In [108]:
df['InvoiceDate'].unique() #Timeseries of number of sales

array(['2010-12-01 08:26:00', '2010-12-01 08:28:00',
       '2010-12-01 08:34:00', ..., '2011-12-09 12:31:00',
       '2011-12-09 12:49:00', '2011-12-09 12:50:00'], dtype=object)

In [109]:
!pip install pycountry
import pycountry #converts country names to country codes demanded by choropleth

df['Country'] = df['Country'].replace({'EIRE':'Ireland', 'Channel Islands':'United Kingdom', 'RSA':'South Africa'})

spent_by_country = df.groupby(by='Country').sum()[['TotalSpent']]
spent_by_country.drop(['European Community', 'Unspecified'], axis=0, inplace=True)

country_list = spent_by_country.index

#getting country codes
countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3
codes = [countries.get(country, 'Unknown') for country in country_list]
codes[7] = 'CZE'
codes[31] = 'USA'

spent_by_country['CountryCode'] = codes
spent_by_country.reset_index(level=0, inplace=True)
spent_by_country.sort_values(by='TotalSpent', ascending=False)

,Country,TotalSpent,CountryCode
33,United Kingdom,7.328842e+06,GBR
21,Netherlands,2.854463e+05,NLD
14,Ireland,2.655459e+05,IRL
11,Germany,2.288671e+05,DEU
10,France,2.090240e+05,FRA
0,Australia,1.385213e+05,AUS
28,Spain,6.157711e+04,ESP
30,Switzerland,5.644395e+04,CHE
3,Belgium,4.119634e+04,BEL
29,Sweden,3.837833e+04,SWE


In [110]:
country_list

Index(['Australia', 'Austria', 'Bahrain', 'Belgium', 'Brazil', 'Canada',
       'Cyprus', 'Czech Republic', 'Denmark', 'Finland', 'France', 'Germany',
       'Greece', 'Iceland', 'Ireland', 'Israel', 'Italy', 'Japan', 'Lebanon',
       'Lithuania', 'Malta', 'Netherlands', 'Norway', 'Poland', 'Portugal',
       'Saudi Arabia', 'Singapore', 'South Africa', 'Spain', 'Sweden',
       'Switzerland', 'USA', 'United Arab Emirates', 'United Kingdom'],
      dtype='object', name='Country')

In [111]:
print('UK sales account for around {}% of the total amount spent on the store'.format(spent_by_country['TotalSpent'][33]/spent_by_country['TotalSpent'].sum()))

UK sales account for around 0.8227775335560462% of the total amount spent on the store


In [112]:
#Proportionate amount spent by customers of each country

import plotly.graph_objects as go

fig = go.Figure()

fig.add_choropleth(locations=spent_by_country['CountryCode'],
                   z=np.log10(spent_by_country['TotalSpent']),
                   text=spent_by_country['Country'],
                   colorscale='cividis',
                   autocolorscale=False,
                   reversescale=False)
fig.show()

In [113]:

len(df['CustomerID'].unique())

4338

In [114]:
#RFM
#Assumptions: the recency of a purchase made up to 11 years ago isn't going to help the analysis very much. Therefore the recency will be calculated using the last invoice date: 10/12/2011.
import datetime as dt

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
customer_df = df[['CustomerID', 'InvoiceNo','InvoiceDate', 'TotalSpent']]
latest_date = customer_df['InvoiceDate'].max() 

#Changing columns to:
#InvoiceDate --> Days since last purchase (as of 10/12/2011). This is the "Recency" part of the RFM analysis.
#InvoiceNo --> n.o. invoices (transactions) per customer. This is the "Frequency" of customer transactions.
#TotalSpent --> Mean of TotalSpent. This is the "Monetary" part of the RFM analysis.
#Quantity --> Total number of items bought by each customer.

customer_df = df.groupby(by='CustomerID').agg({ 'Quantity':'sum', 'InvoiceNo':'count', 'TotalSpent':'mean', 'InvoiceDate':lambda x: (latest_date - x.max()).days })
customer_df.rename(columns = {'InvoiceDate':'Recency', 'InvoiceNo':'Freq', 'TotalSpent':'MeanTotalSpent'}, inplace=True)

In [115]:
customer_df.head()

,Quantity,Freq,MeanTotalSpent,Recency
CustomerID,,,,
12346.0,74215,1,77183.600000,325
12347.0,2458,182,23.681319,1
12348.0,2341,31,57.975484,74
12349.0,631,73,24.076027,18
12350.0,197,17,19.670588,309


In [116]:
customer_df.corr()

,Quantity,Freq,MeanTotalSpent,Recency
Quantity,1.000000,0.430116,0.326191,-0.123758
Freq,0.430116,1.000000,-0.012553,-0.206125
MeanTotalSpent,0.326191,-0.012553,1.000000,0.024617
Recency,-0.123758,-0.206125,0.024617,1.000000


In [117]:
!pip install plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [118]:
fig = make_subplots(rows=1, cols=4, subplot_titles=customer_df.columns + ' Distribution', y_title='Frequency')

fig.add_trace(go.Histogram(x=customer_df['Quantity']), row=1, col=1)
fig.add_trace(go.Histogram(x=customer_df['Freq']), row=1, col=2)
fig.add_trace(go.Histogram(x=customer_df['MeanTotalSpent']), row=1, col=3)
fig.add_trace(go.Histogram(x=customer_df['Recency']), row=1, col=4)

fig['layout']['xaxis']['title']='Quantity Bought Per Customer'
fig['layout']['xaxis2']['title']='Number Of Purchases Per Customer'
fig['layout']['xaxis3']['title']='Mean Total Amount Spent Per Customer (£)'
fig['layout']['xaxis4']['title']='Days Since Last Purchase'

fig.show()

In [119]:
pd.DataFrame(df['Description'].value_counts()).reset_index().rename(columns={"index": "StockCode", "Description": "Count"})

,StockCode,Count
0,WHITE HANGING HEART T-LIGHT HOLDER,2028
1,REGENCY CAKESTAND 3 TIER,1723
2,JUMBO BAG RED RETROSPOT,1618
3,ASSORTED COLOUR BIRD ORNAMENT,1408
4,PARTY BUNTING,1396
...,...,...
3872,DOORKNOB CERAMIC IVORY,1
3873,ROBIN CHRISTMAS CARD,1
3874,MUMMY MOUSE RED GINGHAM RIBBON,1
3875,HAPPY BIRTHDAY CARD TEDDY/CAKE,1


In [120]:
purchases = df[['CustomerID', 'StockCode', 'Quantity']]

table = pd.pivot_table(purchases, values='Quantity', index=['CustomerID', 'StockCode'])
table

Quantity
CustomerID StockCode          
12346.0    23166       74215.0
12347.0    16008          24.0
           17021          36.0
           20665           6.0
           20719          10.0
...                        ...
18287.0    84920           4.0
           85039A         48.0
           85039B         40.0
           85040A         24.0
           85173          48.0

[266792 rows x 1 columns]

In [121]:
#Most purchased item per customer

ix = purchases.groupby('CustomerID')['Quantity'].idxmax()
purchases = purchases.loc[ix, ["CustomerID", "StockCode"]]
purchases_final.rename({'StockCode':'MostBoughtItem'})
purchases_final

,CustomerID,StockCode
61619,12346.0,23166
148290,12347.0,23076
34090,12348.0,21981
485568,12349.0,21231
80327,12350.0,22348
...,...,...
111054,18280.0,22084
222954,18281.0,22037
291855,18282.0,23187
263660,18283.0,23077


In [126]:
#Saving customer data to csv in drive

#customer_df.to_csv('/content/drive/MyDrive/online_retail_data_analysis/customer_df', index=False, encoding='utf-8-sig')

In [ ]:
#What are the customers coming back to buy (most common items in each transaction per customer)?